**Initialize**

Import packages and initialize train and test datasets

In [43]:
import time
import numpy as np
import pandas as pd
import scipy.sparse as sp
from numpy.linalg import norm
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix
from sklearn.metrics import f1_score, accuracy_score
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
import random

random.seed(10)

# read in the train dataset
train_mat = pd.read_csv(
    'train.dat', 
     sep='delimiter', header=None, engine ='python')

# read in the test dataset
test = pd.read_csv(
    'test.dat', 
     sep='delimiter', header=None, engine ='python')

# separate docs and classes
train_vals = train_mat.iloc[:,:].values
train_docs = [n[0][2:] for n in train_vals]
train_cls = [n[0][0] for n in train_vals]

# separate docs and classes
test_vals = test.iloc[:,:].values
test_docs = [n[0][0:] for n in test_vals]
test_cls = [] * len(test_docs)

**Preprocess Data Functions**

In [44]:
def remove_stopwords(docs):
    stop_words = set(stopwords.words('english'))
    punctuation = set(string.punctuation)
    cleaned_docs = []
    for doc in docs:
        words = word_tokenize(doc.lower())
        words = [word for word in words if word not in stop_words and word not in punctuation]
        cleaned_docs.append(" ".join(words))
    return cleaned_docs

def stem_docs(docs):
    stemmer = PorterStemmer()
    stemmed_docs = []
    for doc in docs:
        stemmed_doc = ' '.join(stemmer.stem(word) for word in doc.split())
        stemmed_docs.append(stemmed_doc)
    return stemmed_docs

def lemmatize_docs(docs):
    lemmatizer = WordNetLemmatizer()
    lemmatized_docs = []
    for doc in docs:
        token_list = nltk.word_tokenize(doc)
        pos_tags = nltk.pos_tag(token_list)
        lemmatized_tokens = []
        for token, pos in pos_tags:
            pos_letter = pos[0].lower() if pos[0].lower() in ['a', 'n', 'v', 'r'] else 'n'
            lemma = lemmatizer.lemmatize(token, pos=pos_letter)
            lemmatized_tokens.append(lemma)
        lemmatized_docs.append(" ".join(lemmatized_tokens))
    return lemmatized_docs

**Preprocess Data**

Perform stopword removal on train and test datasets

In [45]:
start = time.time()
train_docs = remove_stopwords(train_docs)
test_docs = remove_stopwords(test_docs)
print("Time to remove stopwords: ", time.time() - start)

Time to remove stopwords:  21.748326301574707


Perform lemmatization on train and test datasets

In [46]:
start = time.time()
train_docs = lemmatize_docs(train_docs)
test_docs = lemmatize_docs(test_docs)
print("Time to lemmatize: ", time.time() - start)

Time to lemmatize:  212.67791604995728


Perform stemming on train and test datasets

In [47]:
# start = time.time()
# train_docs = stem_documents(train_docs)
# test_docs = stem_documents(test_docs)
# print("Time to stem: ", time.time() - start)

Create a subset of the train dataset to find best c, k values using f1_score

In [48]:
train_docs_subset = train_docs[0:2000]
train_cls_subset = train_cls[0:2000]

**Sparse Matrix Functions**

In [49]:
def cmer(doc, c=3):
    r""" Given a name and parameter c, return the vector of c-mers associated with the doc
    """
    doc = doc.lower()
    if len(doc) < c:
        return [doc]
    v = []
    for i in range(len(doc)-c+1):
        v.append(doc[i:(i+c)])
    return v

def build_matrix(docs, idx):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    nrows = len(docs)
    nnz = 0
    for d in docs:
        nnz += len(list(w for w in d if w in idx))
    ncols = len(idx)
        
    # set up memory
    ind = np.zeros(nnz, dtype=int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        cnt = Counter(d)
        keys = list(k for k,_ in cnt.most_common() if k in idx)
        l = len(keys)
        for j,k in enumerate(keys):
            if k in idx:
                ind[j+n] = idx[k]
                val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat

def csr_info(mat, name="", non_empty=False):
    r""" Print out info about this CSR matrix. If non_empty, 
    report number of non-empty rows and cols as well
    """
    if non_empty:
        print("%s [nrows %d (%d non-empty), ncols %d (%d non-empty), nnz %d]" % (
                name, mat.shape[0], 
                sum(1 if mat.indptr[i+1] > mat.indptr[i] else 0 
                for i in range(mat.shape[0])), 
                mat.shape[1], len(np.unique(mat.indices)), 
                len(mat.data)))
    else:
        print( "%s [nrows %d, ncols %d, nnz %d]" % (name, 
                mat.shape[0], mat.shape[1], len(mat.data)) )

def csr_l2normalize(mat, copy=False, **kargs):
    r""" Normalize the rows of a CSR matrix by their L-2 norm. 
    If copy is True, returns a copy of the normalized matrix.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        for j in range(ptr[i], ptr[i+1]):
            rsum += val[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = 1.0/np.sqrt(rsum)
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum
            
    if copy is True:
        return mat
    
def build_idx(train_mat):
    r""" Build a mapping from word to ID and vice versa. 
    """
    idx = {}
    tid = 0
    for d in train_mat:
        for w in d:
            w.lower()
            if w not in idx:
                idx[w] = tid
                tid += 1
    return idx

def textToMatrix(train_docs, test_docs, c=3):
    train_mat = [cmer(l,c) for l in train_docs]
    test_mat = [cmer(l,c) for l in test_docs]
    return train_mat, test_mat

**Classify Functions**

In [50]:
# only used for testing c, k values
def split_data(mat, cls, fold=1, d=10):
    r""" Split the matrix and class info into train and test data using d-fold hold-out
    """
    n = mat.shape[0]
    r = int(np.ceil(n*1.0/d))
    mattr = []
    clstr = []
    # split mat and cls into d folds
    for f in range(d):
        if f+1 != fold:
            mattr.append( mat[f*r: min((f+1)*r, n)] )
            clstr.extend( cls[f*r: min((f+1)*r, n)] )
    # join all fold matrices that are not the test matrix
    train = sp.vstack(mattr, format='csr')
    # extract the test matrix and class values associated with the test rows
    test = mat[(fold-1)*r: min(fold*r, n), :]
    clste = cls[(fold-1)*r: min(fold*r, n)]

    return train, clstr, test, clste

# my knn implementation
def classify(test_row, train_mat, train_cls, k):
    # calc similarities between test_row and training data
    dots = test_row.dot(train_mat.T).todense().tolist()[0]
    similarities = [dots[i] for i in range(len(dots))]

    # if no neighbors were found, choose 1
    if sum(similarities) == 0:
        return random.randint(1, 4)

    # get the indices of the k-nns
    nn_indices = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)[:k]

    # count the occurrence of each class label among the nns
    nn_labels = [train_cls[i] for i in nn_indices]
    label_counts = Counter(nn_labels)

    # if tie, perform a weighted vote based on similarity
    if len(label_counts) > 1 and label_counts.most_common(1)[0][1] == label_counts.most_common(2)[1][1]:
        label_weights = {label: 0 for label in label_counts.keys()}
        for i in nn_indices:
            label_weights[train_cls[i]] += similarities[i]
        return max(label_weights, key=label_weights.get)

    # else, return the majority label
    return label_counts.most_common(1)[0][0]

# only used for testing c, k values
def classify_docs(train_docs, train_cls, c, k, d=10):
    r""" Classify docs using c-mer frequency vector representations of the names and kNN classification with 
    cosine similarity and 10-fold cross validation
    """
    docs = [cmer(n, c) for n in train_docs]
    idx = build_idx(docs)
    mat = build_matrix(docs, idx)
    # since we're using cosine similarity, normalize the vectors
    csr_l2normalize(mat)
    
    f1 = 0.0
    accuracy = 0.0
    macc = 0.0
    for f in range(d):
        # split data into training and testing
        train, clstr, test, clste = split_data(mat, train_cls, f+1, d)
        # predict the class of each test sample
        clspr = [ classify(test[i,:], train, clstr, k) for i in range(test.shape[0]) ]
        # compute the f1, accuracy of the prediction
        acc = 0.0
        for i in range(len(clste)):
            if clste[i] == clspr[i]:
                acc += 1
        acc /= len(clste)
        macc += acc
        f1 += f1_score(clste, clspr, average='macro')
        accuracy += accuracy_score(clste, clspr)
        
    return f1/d, accuracy/d, macc/d

**Test Train Subset**

Get best c, k values by calculating the f1_score of the train subset using c, k values 1-10

In [51]:
best = (0.0, 0.0, 0.0, 0.0, 0.0)
for c in range(1,10+1):
    for k in range(1, 10+1):
        f1, accuracy, macc = classify_docs(train_docs_subset, train_cls_subset, c=c, k=k)
        #print("c=%d, k=%d, %f, %f, %f" % (c, k, f1, accuracy, macc))
        if f1 > best[2]:
            best = (c, k, f1, accuracy, macc)
#print("Best: c=%d, k=%d, %f, %f, %f" % best)

c = best[0]
k = best[1]

**Build Matrices**

Build and normalize matrices

In [52]:
train_mat, test_mat = textToMatrix(train_docs, test_docs, c)
idx = build_idx(train_mat)

train_mat = csr_l2normalize(build_matrix(train_mat, idx), copy=True)
test_mat = csr_l2normalize(build_matrix(test_mat, idx), copy=True)

csr_info(train_mat, "train_mat")
csr_info(test_mat, "test_mat")

train_mat [nrows 102080, ncols 129391, nnz 15727513]
test_mat [nrows 25520, ncols 129391, nnz 3922131]


**Run Predictions**

Predict classifications on test dataset

In [53]:
test_cls = [ classify(test_mat[i,:], train_mat, train_cls, k) for i in range(test_mat.shape[0]) ]

**Output**

Output predictions to file

In [54]:
output_file = open('output.dat', 'w', newline='')
predictions = pd.Series(test_cls)
predictions.to_csv(output_file, index=False, header=None)

Output predictions to file (other version not always recording every row)

In [55]:
with open('yayaya.dat', 'w') as f:
    for item in test_cls:
        f.write(f"{item}\n")